# The interesting topics in this year

In [1]:
import pandas as pd

def filter_by_topic(df, topic):
    df['google_scholar_link'] = df['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))
    df = df.drop_duplicates(subset=['title'])
    return df[df['topic'].str.contains(topic, case=False, na=False)].copy()

In [2]:
from semanticscholar import SemanticScholar
import time
from tqdm import tqdm
tqdm.pandas()

def get_paper_citation(title):
    sch = SemanticScholar()
    time.sleep(1)
    title = title.replace('-', ' ')
    try:
        papers = sch.search_paper(title)
        if len(papers) > 0:
            return papers[0].citationCount
    except:
        print(f'Error in get citation for {title}')
    
    return 0
    

def get_topic_citations(df):
    df['ss_citations'] = df['title'].progress_apply(get_paper_citation)
    return df

In [3]:
def get_conf_topics(conf_path='data/2023/5_iclr.csv_topics.csv', topic='multimodal'):
    df_conf = pd.read_csv(conf_path)
    df_conf_topic = filter_by_topic(df_conf, topic=topic)
    print(len(df_conf_topic), len(df_conf))
    df_conf_topic = get_topic_citations(df_conf_topic)
    return df_conf_topic[['title','year','source','ss_citations','google_scholar_link']]

In [4]:
def get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal']):
    df_conf = pd.read_csv(conf_path)

    save_path = conf_path + '_emerging.xlsx'
    writer = pd.ExcelWriter(save_path, engine="xlsxwriter")

    for topic in topics:
        print(f"topic {topic}")
        df_conf_topic = get_conf_topics(conf_path, topic)
        df_conf_topic.to_excel(writer, sheet_name=topic, index=False)
    writer.close()
    print(f'Saved to {save_path}')

## 2023, ICLR, May

In [5]:
# get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal', 'diffusion models', 'llm', 'self-supervision'])

## 2023, CVPR, June

In [6]:
get_conf_topics_all(conf_path='data/2023/6_cvpr.csv_topics.csv', topics=['3d', 'zero_few-shot', 'transformer', 'segmentation', 'multimodal', 'imitation learning', 'radiance field', 'motion', 'diffusion models', 'multi-agent'])

topic 3d
270 2359


  2%|▏         | 5/270 [00:07<07:31,  1.70s/it]

Error in get citation for MarS3D: A Plug and Play Motion Aware Model for Semantic Segmentation on Multi Scan 3D Point Clouds


  9%|▉         | 25/270 [00:49<08:40,  2.13s/it]

Error in get citation for 3D Spatial Multimodal Knowledge Accumulation for Scene Graph Prediction in Point Cloud


...

topic multi-agent
6 2359


100%|██████████| 6/6 [00:11<00:00,  2.15s/it]

Error in get citation for StarCraftImage: A Dataset for Prototyping Spatial Reasoning Methods for Multi Agent Environments


100%|██████████| 6/6 [00:14<00:00,  2.39s/it]

Saved to data/2023/6_cvpr.csv_topics.csv_emerging.xlsx
